In [9]:
# import these first.
import copy
import getpass
import os
from datetime import datetime
import logging
import fdi.dataset.serializable
from fdi.dataset.product import Product
from fdi.dataset.metadata import Parameter, NumericParameter, MetaData
from fdi.dataset.finetime import FineTime1, utcobj
from fdi.dataset.dataset import ArrayDataset, TableDataset, Column
from fdi.dataset.composite import Composite
from fdi.pal.context import Context, MapContext
from fdi.pal.productref import ProductRef
from fdi.pal.query import MetaQuery, StorageQuery
from fdi.pal.poolmanager import PoolManager, DEFAULT_MEM_POOL
from fdi.pal.productstorage import ProductStorage
from fdi.dataset.serializable import serializeClassID
from fdi.dataset.deserialize import deserializeClassID

In [46]:
# Creation:
x = Product(description="product example with several datasets",
            instrument="Crystal-Ball", modelName="Mk II")
x.meta['description'].value  # == "product example with several datasets"
# ways to add datasets
i0 = 6
i1 = [[1, 2, 3], [4, 5, i0], [7, 8, 9]]
i2 = 'ev'                 # unit
i3 = 'image1'     # description
image = ArrayDataset(data=i1, unit=i2, description=i3)
x["RawImage"] = image
x["RawImage"].data  # == [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
# no unit or description. different syntax but same function as above
x.set('QualityImage', ArrayDataset(
    [[0.1, 0.5, 0.7], [4e3, 6e7, 8], [-2, 0, 3.1]], unit=i2))
x["QualityImage"].unit  # is None
# add a tabledataset
s1 = [('col1', [1, 4.4, 5.4E3], 'eV'),
      ('col2', [[0, 43.2, 2E3],[0,4, 22E2]], 'cnt')]
x["Spectrum"] = TableDataset(data=s1)

x.creator='Yuxin'

# x1 = Product(description='in store')
# add a tabledataset
s1 = [('energy', [1, 4.4, 5.6], 'eV'), ('freq', [0, 43.2, 2E3], 'Hz')]
x1["Spectrum"] = TableDataset(data=s1)

In [42]:
# data = serializeClassID(x)
# print(type(data))
data='{"_sets":{"data":{"MyArrayDataset":{"typecode":"herschel_dataset.ia.dataset.ArrayDataset","default":"null","unit":"eV [1 eV = 1.602176462E-19 J]","classID":"ArrayDataset","data":[0.0,100.0,200.0,300.0,400.0,500.0,600.0,700.0,800.0,900.0],"meta":{"_sets":{"classID":"ODict","data":{"a":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter"},"usefor":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter"}}},"classID":"MetaData","listeners":[]},"description":"a description"}}},"classID":"Product","listeners":[],"meta":{"_sets":{"classID":"ODict","data":{"modelName":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Model name attached to this product"},"creator":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Generator of this product"},"endDate":{"valid":"null","typecode":"herschel.share.fltdyn.time.FineTime","classID":"StringParameter","endDate":{"classID":"FineTime","format":"2020-09-09T09:20:25.283000 TAI (1978334425283000)","tai":1978334425283000},"description":"End date of this product"},"description":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Name of this product"},"instrument":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Instrument attached to this product"},"type":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Product Type Identification"},"creationDate":{"valid":"null","typecode":"herschel.share.fltdyn.time.FineTime","classID":"StringParameter","description":"Creation date of this product","creationDate":{"classID":"FineTime","format":"2020-09-09T09:20:25.179000 TAI (1978334425179000)","tai":1978334425179000}},"formatVersion":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Version of product format"},"startDate":{"valid":"null","typecode":"herschel.share.fltdyn.time.FineTime","classID":"StringParameter","description":"Start date of this product","startDate":{"classID":"FineTime","format":"2020-09-09T09:20:25.179000 TAI (1978334425179000)","tai":1978334425179000}}}},"classID":"MetaData","listeners":[]},"history":{"_sets":{"classID":"ODict","data":{}},"classID":"History","PARAM_HISTORY":"","meta":{"_sets":{"classID":"ODict","data":{}},"classID":"MetaData","listeners":[]},"HIST_SCRIPT":"","TASK_HISTORY":"","description":"UNKNOWN"}}'
x = deserializeClassID(data)
print(x)

2462
{meta = "MetaData{[modelName = StringParameter{ "None", "Model name attached to this product", dflt , vld null tcode=java.lang.String}, creator = StringParameter{ "None", "Generator of this product", dflt , vld null tcode=java.lang.String}, endDate = StringParameter{ "None", "End date of this product", dflt , vld null tcode=herschel.share.fltdyn.time.FineTime}, description = StringParameter{ "None", "Name of this product", dflt , vld null tcode=java.lang.String}, instrument = StringParameter{ "None", "Instrument attached to this product", dflt , vld null tcode=java.lang.String}, type = StringParameter{ "None", "Product Type Identification", dflt , vld null tcode=java.lang.String}, creationDate = StringParameter{ "None", "Creation date of this product", dflt , vld null tcode=herschel.share.fltdyn.time.FineTime}, formatVersion = StringParameter{ "None", "Version of product format", dflt , vld null tcode=java.lang.String}, startDate = StringParameter{ "None", "Start date of this prod

In [47]:
data = serializeClassID(x)
print(data)
# data='{"_sets":{"data":{"MyArrayDataset":{"typecode":"herschel_dataset.ia.dataset.ArrayDataset","default":"null","unit":"eV [1 eV = 1.602176462E-19 J]","classID":"ArrayDataset","data":[0.0,100.0,200.0,300.0,400.0,500.0,600.0,700.0,800.0,900.0],"meta":{"_sets":{"classID":"ODict","data":{"a":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","value":"value1"},"usefor":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","value":"your mom"}}},"classID":"MetaData","listeners":[]},"description":"a description"}}},"classID":"Product","listeners":[],"meta":{"_sets":{"classID":"ODict","data":{"modelName":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Model name attached to this product","value":"Unknown"},"creator":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Generator of this product","value":"Your daddy Yuxin"},"endDate":{"valid":"null","typecode":"herschel.share.fltdyn.time.FineTime","classID":"StringParameter","description":"End date of this product","value":"DATE"},"description":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Name of this product","value":"Results of Test session"},"instrument":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Instrument attached to this product","value":"Unknown"},"type":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Product Type Identification","value":"Unknown"},"creationDate":{"valid":"null","typecode":"herschel.share.fltdyn.time.FineTime","classID":"StringParameter","description":"Creation date of this product","value":"DATE"},"formatVersion":{"valid":"null","typecode":"java.lang.String","classID":"StringParameter","description":"Version of product format","value":"1.0"},"startDate":{"valid":"null","typecode":"herschel.share.fltdyn.time.FineTime","classID":"StringParameter","description":"Start date of this product","value":"DATE"}}},"classID":"MetaData","listeners":[]},"history":{"_sets":{"classID":"ODict","data":{}},"classID":"History","PARAM_HISTORY":"","meta":{"_sets":{"classID":"ODict","data":{}},"classID":"MetaData","listeners":[]},"HIST_SCRIPT":"","TASK_HISTORY":"","description":"UNKNOWN"}}'
# print(len(data))
x = deserializeClassID(data)
# print(x)

Seriablizable: cls-->
<class 'fdi.dataset.serializable.SerializableEncoder'>
{"meta": {"_sets": {"data": {"description": {"description": "Description of this product", "value": "product example with several datasets", "default": "UNKOWN", "valid": null, "typecode": "B", "classID": "StringParameter"}, "type": {"description": "Product Type identification. Fully qualified Python class name or CARD.", "value": "Product", "default": "BaseProduct", "valid": null, "typecode": "B", "classID": "StringParameter"}, "creator": {"description": "Generator of this product. Example name of institute, organization, person, software, special algorithm etc.", "value": "Yuxin", "default": "UNKOWN", "valid": null, "typecode": "B", "classID": "StringParameter"}, "creationDate": {"description": "Creation date of this product", "value": {"tai": 0, "format": "%Y-%m-%dT%H:%M:%S.%f", "classID": "FineTime1"}, "default": 0, "valid": null, "typecode": "%Y-%m-%dT%H:%M:%S.%f", "classID": "DateParameter"}, "rootCause"

In [3]:
### a pool for demonstration will be create here
### TEST FOR HTTPCLIENTPOOL
demopoolpath = '/yuxin_test1'
demopool = 'http://192.168.1.11:5000' + demopoolpath
# clean possible data left from previous runs
# os.system('rm -rf ' + demopoolpath)
PoolManager.getPool(DEFAULT_MEM_POOL).removeAll()
PoolManager.removeAll()
urn = 'urn:' + demopool + ':fdi.dataset.product.Product:0'
print(urn)
pstore = ProductStorage(pool=demopool)
# pstore.getPool('http://192.168.1.11:5000/yuxin_test/fdi').loadProduct(urn)
pstore.save(x1)
# pstore.getPools()

urn:http://192.168.1.11:5000/yuxin_test1:fdi.dataset.product.Product:0
READ HK FROM REMOTE===>poolurl: http://192.168.1.11:5000/yuxin_test1
GET REQUEST API: http://192.168.1.11:5000/v0.6/yuxin_test1/hk/classes
GET REQUEST API: http://192.168.1.11:5000/v0.6/yuxin_test1/hk/tags
GET REQUEST API: http://192.168.1.11:5000/v0.6/yuxin_test1/hk/urns
POST API: http://192.168.1.11:5000/v0.6/yuxin_test1/fdi.dataset.product.Product/0


ProductRef{ ProductURN=urn:http://192.168.1.11:5000/yuxin_test1:fdi.dataset.product.Product:0, meta=MetaData{[description = Parameter{ in store <string>, "Description of this product"}, type = Parameter{ Product <string>, "Product Type identification. Fully qualified Python class name or CARD."}, creator = Parameter{ UNKOWN <string>, "Generator of this product. Example name of institute, organization, person, software, special algorithm etc."}, creationDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Creation date of this product"}, rootCause = Parameter{ UNKOWN <string>, "Reason of this run of pipeline."}, schema = Parameter{ 0.3 <string>, "Version of product schema"}, startDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal start time  of this product."}, endDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal end time  of this product."}, instrument = Parameter{ UNKOWN <string>, "Instrument that generated data of this product"

In [17]:
# a pool for demonstration will be create here
#demopoolpath = '/tmp/demopool_' + getpass.getuser()
demopoolpath = '/demopool_' + getpass.getuser()
demopool = 'file://' + demopoolpath
# demopool = 'http://127.0.0.1:5000' + demopoolpath
# clean possible data left from previous runs
# os.system('rm -rf ' + demopoolpath)
PoolManager.getPool(DEFAULT_MEM_POOL).removeAll()
PoolManager.removeAll()

# create a prooduct and save it to a pool
x1 = Product(description='in store')
# add a tabledataset
s1 = [('energy', [1, 4.4, 5.6], 'eV'), ('freq', [0, 43.2, 2E3], 'Hz')]
x1["Spectrum"] = TableDataset(data=s1)
# create a product store
pstore = ProductStorage(pool=demopool)
print(pstore.save(x1,tag='ABC'))
print(pstore.getPools())
print(pstore.getPool('file:///demopool_teasyu').schematicLoadProduct('fdi.dataset.product.Product', '0'))
q = MetaQuery(Product, 'm["description"] == "in store"')
res = pstore.select(q)

# pstore.save(x1)

ProductRef{ ProductURN=urn:file:///demopool_teasyu:fdi.dataset.product.Product:0, meta=MetaData{[description = Parameter{ in store <string>, "Description of this product"}, type = Parameter{ Product <string>, "Product Type identification. Fully qualified Python class name or CARD."}, creator = Parameter{ UNKOWN <string>, "Generator of this product. Example name of institute, organization, person, software, special algorithm etc."}, creationDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Creation date of this product"}, rootCause = Parameter{ UNKOWN <string>, "Reason of this run of pipeline."}, schema = Parameter{ 0.3 <string>, "Version of product schema"}, startDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal start time  of this product."}, endDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal end time  of this product."}, instrument = Parameter{ UNKOWN <string>, "Instrument that generated data of this product"}, modelName 

In [20]:
# urn = 'urn:' + demopool + ':fdi.dataset.product.Product:0'
# pstore.load(urn)
#pstore.register('file:///pool_teasyu')
print(pstore.getPools())
del pstore._pools['file:///pool_teasyu']
pstore.getPools()

['file:///demopool_teasyu', 'file:///pool_teasyu']


['file:///demopool_teasyu']

In [1]:
from fdi.pns.fdi_requests import *
#urn2fdiurl("urn:http://192.168.1.11:5000/tmp/a/b:fdi.dataset.product.Product:0", contents='product', method='POST')
# urn2fdiurl("http://192.168.1.11:5000/tmp/a/b", contents='housekeeping')
urn2fdiurl(urn="http://192.168.1.11:5000/tmp/a/b", contents='classes')

'http://192.168.1.11:5000/v0.6/tmp/a/b/hk/classes'

In [8]:
pstore.getPools()
pstore.register('file:///tmp/demopool_teasyu')

In [15]:
# get the urn string
urn = prodref.urn
# urn:file:///tmp/demopool_mh:fdi.dataset.product.Product:0

urn::///tmp/yuxin_test:fdi.dataset.product.Product:1


In [73]:
import requests
from requests.auth import HTTPBasicAuth
#x = requests.get('http://localhost:5000/v0.6/httppool', auth=HTTPBasicAuth('gsegment', '1234567'))
#print(x.text)
with open('/tmp/yuxin_test/fdi.dataset.product.Product_1', 'rb') as fp:
    content = fp.read()
x = requests.post('http://localhost:5000/v0.6/httppool/yuxin_test1', auth=HTTPBasicAuth('gsegment', '123456'), headers=headers, data=content)
print(x.text)

{"result": "urn:yuxin_test1/fdi:fdi.dataset.product.Product:4", "msg": "save to file:///data/yuxin_test1/fdi with successfully.", "timestamp": 1593680256.3668618}


In [58]:
urn = 'urn:file:///tmp/yuxin_test/fdi.dataset.product.Product:fdi.dataset.product.Product:0'
client = urn.replace('file://' + '/tmp/', '')
client_fields = client.split(':')
urn = client_fields[0] + ':' + 'file://' + '/tmp/' + client_fields[1] + ':'+ client_fields[2]+ ':' + client_fields[3]
urn

'urn:file:///tmp/yuxin_test/fdi.dataset.product.Product:fdi.dataset.product.Product:0'

In [8]:
# x = requests.get('http://localhost:5000/v0.6/httppool/yuxin_test1', auth=HTTPBasicAuth('gsegment', '123456'))
# x = requests.delete('http://localhost:5000/v0.6/httppool/yuxin_test1', auth=HTTPBasicAuth('gsegment', '123456'))
q = MetaQuery(Product, 'm["creator"] == "Yuxin"')
data = {"type": "MetaQuery", "where": q.getWhere(), "allVersions": q.retrieveAllVersions(), "variable": q.getVariable()}
query = json.dumps(data)
x = requests.post('http://localhost:5000/v0.6/httppool/query', auth=HTTPBasicAuth('gsegment', '123456'), data=query)

NameError: name 'requests' is not defined

In [129]:
# b= json.dumps(data)
# a = deserializeClassID(b)
type(q.getVariable())

str

In [6]:
# clean possible data left from previous runs
defaultpoolpath = '/tmp/pool_' + getpass.getuser()
newpoolpath = '/tmp/newpool_' + getpass.getuser()
os.system('rm -rf ' + defaultpoolpath)
os.system('rm -rf ' + newpoolpath)
PoolManager.getPool(DEFAULT_MEM_POOL).removeAll()
PoolManager.removeAll()
# make a productStorage
defaultpool = 'file://'+defaultpoolpath
pstore = ProductStorage(defaultpool)
# make another
newpoolname = 'file://' + newpoolpath
pstore2 = ProductStorage(newpoolname)
# add some products to both storages
n = 7
for i in range(n):
    a0, a1, a2 = 'desc %d' % i, 'fatman %d' % (i*4), 5000+i
    if i < 3:
        x = Product(description=a0, instrument=a1)
        x.meta['extra'] = Parameter(value=a2)
    elif i < 5:
        x = Context(description=a0, instrument=a1)
        x.meta['extra'] = Parameter(value=a2)
        x = MapContext(description=a0, instrument=a1)
        x.meta['extra'] = Parameter(value=a2)
        x.meta['time'] = Parameter(value=FineTime1(a2))
    if i < 4:
        r = pstore.save(x)
    else:
        r = pstore2.save(x)
    print(r.urn)
# Two pools, 7 products
pstore.register(newpoolname)

urn:file:///tmp/pool_teasyu:fdi.dataset.product.Product:0
urn:file:///tmp/pool_teasyu:fdi.dataset.product.Product:1
urn:file:///tmp/pool_teasyu:fdi.dataset.product.Product:2
urn:file:///tmp/pool_teasyu:fdi.pal.context.MapContext:0
urn:file:///tmp/newpool_teasyu:fdi.pal.context.MapContext:0
urn:file:///tmp/newpool_teasyu:fdi.pal.context.MapContext:1
urn:file:///tmp/newpool_teasyu:fdi.pal.context.MapContext:2


In [185]:
# register the newpool above to the  1st productStorage
# pstore.register('file:///data/yuxin_test1/fdi/')
# pstore.wipePool('file:///data/yuxin_test1/fdi/')
print(pstore.getPools())   # == 2
print(pstore._pools)
# make a query on product metadata, which is the variable 'm'
# in the query expression, i.e. ``m = product.meta; ...``
# But '5000 < m["extra"]' does not work. see tests/test.py.
q = MetaQuery(Product, 'm["extra"] == 5004')
# search all pools registered on pstore
res = pstore.select(q)
# [2,3,4,5]
len(res)   # == 4
# [r.product.description for r in res]
res

['file:///tmp/pool_teasyu', 'file:///tmp/newpool_teasyu']
OD{'file:///tmp/pool_teasyu':LocalPool { pool= file:///tmp/pool_teasyu }, 'file:///tmp/newpool_teasyu':LocalPool { pool= file:///tmp/newpool_teasyu }}


[ProductRef{ ProductURN=urn:file:///tmp/newpool_teasyu:fdi.pal.context.MapContext:0, meta=MetaData{[description = Parameter{ desc 4 <string>, "Description of this product"}, type = Parameter{ Product <string>, "Product Type identification. Fully qualified Python class name or CARD."}, creator = Parameter{ UNKOWN <string>, "Generator of this product. Example name of institute, organization, person, software, special algorithm etc."}, creationDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Creation date of this product"}, rootCause = Parameter{ UNKOWN <string>, "Reason of this run of pipeline."}, schema = Parameter{ 0.3 <string>, "Version of product schema"}, startDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal start time  of this product."}, endDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal end time  of this product."}, instrument = Parameter{ fatman 16 <string>, "Instrument that generated data of this product"}, modelN

In [5]:
pstore.save2httppool(product=x, host='abc')
# pstore.getPool('file:///data/yuxin_test')
# dir(ProductStorage)

unable to save to the writable pool.


HTTP POOL URN: abc/tmp/demopool_teasyu


AttributeError: 'LocalPool' object has no attribute 'saveProduct2HttpPool'

In [28]:
pstore.getPools()
pstore.getPool('file:///tmp/demopool_teasyu')._poolurn

'file:///tmp/demopool_teasyu'

In [14]:
from urllib.parse import urlparse
x = urlparse('file:///demopool')
x

ParseResult(scheme='file', netloc='', path='/demopool', params='', query='', fragment='')

In [21]:
auth_user = 'gsegment'
auth_pass = '123456'
baseurl = 'http://192.168.1.11:5000/v0.6/httppool'

# HTTPPOOL >  GET
def httppool_get_pstore():
    x = requests.get(baseurl, auth=HTTPBasicAuth(auth_user, auth_pass))
    print(x.text)
#httppool_get_pstore()

# HTTPPOOL/POOLID > GET
def httppool_get_pool(poolid, resourcename=None, indexstr=None):
    if resourcename and indexstr:
        x = requests.get(baseurl +'/'+ poolid + '?'+ 'resourcename=' +resourcename + '&' + 'indexstr=' + indexstr, auth=HTTPBasicAuth(auth_user, auth_pass))
    else:
        x = requests.get('http://localhost:5000/v0.6/httppool/'+ poolid, auth=HTTPBasicAuth(auth_user, auth_pass))
    print(x.text)
poolid = 'yuxin_test'
#httppool_get_pool(poolid=poolid)
#httppool_get_pool(poolid=poolid, resourcename='fdi.dataset.product.Product', indexstr='2')

# HTTPPOOL/POOLID > DELETE
def httppool_delete_pool(poolid):
    x = requests.delete('http://localhost:5000/v0.6/httppool/'+ poolid, auth=HTTPBasicAuth(auth_user, auth_pass))
    print(x.text)
    
poolid = 'yuxin_test'
# httppool_delete_pool(poolid)

# HTTPPOOL/POOLID > POST
def httppool_post_pool(prod_obj, poolid):
    x = requests.post('http://localhost:5000/v0.6/httppool/' + poolid, \
                      auth=HTTPBasicAuth(auth_user, auth_pass), data=prod_obj)
    print(x.text)
    
def test_post():
    n = 7
    for i in range(n):
        a0, a1, a2 = 'desc %d' % i, 'fatman %d' % (i*4), 5000+i
        if i < 3:
            x = Product(description=a0, instrument=a1)
            x.meta['extra'] = Parameter(value=a2)
        elif i < 5:
            x = Context(description=a0, instrument=a1)
            x.meta['extra'] = Parameter(value=a2)
            x = MapContext(description=a0, instrument=a1)
            x.meta['extra'] = Parameter(value=a2)
            x.meta['time'] = Parameter(value=FineTime1(a2))
        if i < 4:
            obj = serializeClassID(x)
            httppool_post_pool(obj, 'yuxin_test')
            
#test_post()
# HTTPPOOL/QUERY POST
def query_httppool(q):
    data = {"type": "MetaQuery", "where": q.getWhere(), "allVersions": q.retrieveAllVersions(), "variable": q.getVariable()}
    query = json.dumps(data)
    x = requests.post('http://localhost:5000/v0.6/httppool/query', \
                      auth=HTTPBasicAuth(auth_user, auth_pass), data=query)
    print(x.text)
    return x
q = MetaQuery(Product, 'm["extra"] > 5001 and m["extra"] <= 5005')
#r = query_httppool(q)
#len(deserializeClassID(r.text))

tuple

In [2]:
import filelock
import time
    
with filelock.FileLock('/tmp/locks/yuxin_test/lock'):
    for i in range(0, 10):
        print('locked file....')
        time.sleep(5)
        

locked file....
locked file....
locked file....
locked file....
locked file....
locked file....
locked file....
locked file....
locked file....
locked file....


In [2]:
# TEST degree in local pool
# if a product is like: /poolid and an other is like : /poolid/a/b

# clean possible data left from previous runs
defaultpoolpath = '/pool_' + getpass.getuser()+ '/a'
newpoolpath = '/pool_' + getpass.getuser() + '/b/c'
#os.system('rm -rf ' + defaultpoolpath)
#os.system('rm -rf ' + newpoolpath)
PoolManager.getPool(DEFAULT_MEM_POOL).removeAll()
PoolManager.removeAll()
# make a productStorage
defaultpool = 'file://'+defaultpoolpath
pstore = ProductStorage(defaultpool)
# make another
newpoolname = 'file://' + newpoolpath
pstore2 = ProductStorage(newpoolname)

# add some products to both storages
n = 7
for i in range(n):
    a0, a1, a2 = 'desc %d' % i, 'fatman %d' % (i*4), 5000+i
    if i < 3:
        x = Product(description=a0, instrument=a1)
        x.meta['extra'] = Parameter(value=a2)
    elif i < 5:
        x = Context(description=a0, instrument=a1)
        x.meta['extra'] = Parameter(value=a2)
        x = MapContext(description=a0, instrument=a1)
        x.meta['extra'] = Parameter(value=a2)
        x.meta['time'] = Parameter(value=FineTime1(a2))
    if i < 4:
        r = pstore.save(x)
    else:
        r = pstore2.save(x)
    print(r.urn)
# Two pools, 7 products
# [P P P C] [C M M]

urn:file:///pool_teasyu/a:fdi.dataset.product.Product:0
urn:file:///pool_teasyu/a:fdi.dataset.product.Product:1
urn:file:///pool_teasyu/a:fdi.dataset.product.Product:2
urn:file:///pool_teasyu/a:fdi.pal.context.MapContext:0
urn:file:///pool_teasyu/b/c:fdi.pal.context.MapContext:0
urn:file:///pool_teasyu/b/c:fdi.pal.context.MapContext:1
urn:file:///pool_teasyu/b/c:fdi.pal.context.MapContext:2


In [5]:
# pstore.getPools()
# register the new pool above to the  1st productStorage
pstore.register(newpoolname)
print(len(pstore.getPools()))   # == 2
print(pstore.getPools())
pstore.getPool('file:///pool_teasyu/a').schematicLoadProduct('fdi.pal.context.MapContext', '0')

2
['file:///pool_teasyu/a', 'file:///pool_teasyu/b/c']


{meta = "MetaData{[description = Parameter{ desc 3 <string>, "Description of this product"}, type = Parameter{ Product <string>, "Product Type identification. Fully qualified Python class name or CARD."}, creator = Parameter{ UNKOWN <string>, "Generator of this product. Example name of institute, organization, person, software, special algorithm etc."}, creationDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Creation date of this product"}, rootCause = Parameter{ UNKOWN <string>, "Reason of this run of pipeline."}, schema = Parameter{ 0.3 <string>, "Version of product schema"}, startDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal start time  of this product."}, endDate = Parameter{ 2017-01-01T00:00:00.000000 TAI(0) <finetime>, "Nominal end time  of this product."}, instrument = Parameter{ fatman 12 <string>, "Instrument that generated data of this product"}, modelName = Parameter{ UNKOWN <string>, "Model name of the instrument of this product"}

In [10]:
# make a query on product metadata, which is the variable 'm'
# in the query expression, i.e. ``m = product.meta; ...``
# But '5000 < m["extra"]' does not work. see tests/test.py.
q = MetaQuery(Product, 'm["extra"] > 5001 and m["extra"] <= 5005')
# search all pools registered on pstore
res = pstore.select(q)
# [2,3,4,5]
print(len(res))   # == 4
[r.product.description for r in res]

2


['desc 2', 'desc 3']

In [16]:
alldirs=[]
path = '/data'
def getfiledir(filepath):
    paths = filepath.split('/')
    dirpath = ''
    for i in paths[2:-1]:
        dirpath = dirpath + i + '/'
    return dirpath[0:-1]
def getallpools(path):
    allfilelist=os.listdir(path)
    for file in allfilelist:
        filepath=os.path.join(path,file)
        if os.path.isdir(filepath):
            getallpools(filepath)
        else:
            dirpath = getfiledir(filepath)
            if dirpath not in alldirs:
                alldirs.append(dirpath)
    return alldirs

print(getallpools(path))

['pool_default', 'subclasses/b', 'subclasses/a', 'post_test_pool']
